In [1]:
from bs4 import BeautifulSoup
import requests,json, time
from requests_oauthlib import OAuth1Session
from requests.exceptions import ConnectionError, ReadTimeout, SSLError
import numpy as np
import numpy.random as rd
import MeCab as mc
from collections import defaultdict
import cPickle as pickle
import traceback
from datetime import datetime as dt

IPADIC_NEOLOGD_PATH = '/usr/local/lib/mecab/dic/mecab-ipadic-neologd/'

def unpickle(filename):
    with open(filename, 'rb') as fo:
        p = pickle.load(fo)
    return p

def to_pickle(filename, obj):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, -1)

In [ ]:
class Category():
    def __init__(self, name="", url=""):
        self.name = name
        self.url = url
        self.article_list = []
        
    def addArticle(self, article):
        self.article_list.append(article)
        
class Article():
    def __init__(self, title="", contents=u"未取得", url=""):
        self.url = url
        self.title = title
        self.contents = contents
        self.mecabed_contents = {}
        
    def add_contents(self, contents):
        self.contents = contents
        
    def exec_mecab(self):
        self.mecabed_contents = Article.mecab_analysis(self.contents)
        
    @staticmethod
    def mecab_analysis(sentence):
        t = mc.Tagger('-Ochasen -d {}'.format(IPADIC_NEOLOGD_PATH))
        sentence = sentence.replace('\n', ' ')
        text = sentence.encode('utf-8') 
        node = t.parseToNode(text) 
        ret_list = []
        while node.next: 
            if node.surface != "":  # ヘッダとフッタを除外
                word_type = node.feature.split(",")[0]
                if word_type in ["名詞", "形容詞", "動詞"]:
                    plain_word = node.feature.split(",")[6]
                    if plain_word !="*":
                        ret_list.append(plain_word.decode('utf-8'))
            node = node.next
        return ret_list 

DEBUG = True
class YahooHeadlines():
    def __init__(self):
        self.url = 'http://headlines.yahoo.co.jp/rss/list'
        self.category_list = []
        self.f = open('log/log_{}.log'.format(dt.now().strftime('%Y%m%d_%H%M%S')), 'a+')
        
    def close(self):
        self.f.close()
                      
    def logging(self, log):
        self.f.write(log.encode('utf-8'))
    
    def unpickle(self, filename):
        with open(filename, 'rb') as fo:
            p = pickle.load(fo)
        self.category_list = p

    def pickle(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.category_list, f, -1)
        
    def download_contents(self):
        self.get_category_url_list()
        self.get_article_title_list()
        self.get_all_article()

    def get_url_list(self):
        return self._url_list
    
    def set_category_list(self, category_list):
        self.category_list = category_list
    
    def get_category_list(self):
        return self.category_list
    
    def get_category_url_list(self):
        res = requests.get(self.url)
        news_all = BeautifulSoup(res.text, "xml")
        for link in news_all.find_all('a'):
            url = link.get('href')
            if 'xml' in url and 'my.yahoo.co.jp' not in url:
                self.category_list.append(Category(name=link.parent.text.replace('\n',''), url=url))
        if DEBUG:
            print "len(self.category_list)", len(self.category_list)

    def get_article_title_list(self):
        for category in self.category_list:
            res = requests.get(category.url)
            soup = BeautifulSoup(res.text, "xml")
            for item in soup.find_all('item'):
                category.addArticle(Article(title=item.title.getText(), url=item.link.getText()))

    def count(self):
        print "len(self.category_list)", len(self.category_list)
        for cat in self.category_list:
            print "len(cat.article_list)", len(cat.article_list)

        
    def get_all_article(self, start=0, end=None):
        end = len(self.category_list) if end is None else end
        for cat in self.category_list[start:end]:
            print cat.name
            for article in cat.article_list:
                try:
                    print article.title
                    time.sleep(0.5)   # interval time for reducing server load
                    res = requests.get(article.url)
                    soup = BeautifulSoup(res.text, "xml")
                    t = soup.find("p", "ynDetailText")
                    if len(t.getText()) > 0:
                        temp = []
                        for line in t.prettify().split('\n'):
                            if '<!-- /.paragraph -->' in line:
                                break
                            temp.append( line )
                        article.add_contents(BeautifulSoup("\n".join(temp), "xml").get_text().replace(' ','').replace('\n',''))
                        article.exec_mecab()
                except Exception as e:
                    print "error."
                    self.logging(u"{},{}".format(article.url, article.title))
                    self.logging(traceback.format_exc())

    def export(self):
        news_list = []
        for c in self.category_list:
            for a in c.article_list:
                if u'未取得' != a.contents:
                    news_list.append(a.mecabed_contents)
        return news_list
        

In [ ]:
yh = YahooHeadlines()
print "YahooHeadlines() created."

In [ ]:
yh.get_category_url_list()
print "get_category_url_list() finished."
yh.get_article_title_list()
print "get_article_title_list() finished."

In [ ]:
yh.get_all_article(start=9, end=30)

In [ ]:
dat = yh.export()
to_pickle('mecabed_contents.npy', dat)


In [2]:
dat = unpickle('mecabed_contents.npy')

In [3]:
import os, sys
import pandas as pd
import numpy as np
from datetime import datetime as dt
print "loading PySpark setting..."
spark_home = os.environ.get('SPARK_HOME', None)
print spark_home
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

loading PySpark setting...
/usr/local/bin/spark-1.5.0-bin-hadoop2.6
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.0
      /_/

Using Python version 2.7.10 (default, May 28 2015 17:04:42)
SparkContext available as sc, HiveContext available as sqlContext.


In [4]:
import pandas as pd
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

hashingTF = HashingTF()
documents = sc.parallelize(dat)
def hashing(x):
    return hashingTF.transform([x]).indices[0]

hashed = documents.flatMap(lambda line: line) \
             .map(lambda word: (hashing(word), word)).distinct()

hashed_word = pd.DataFrame(hashed.collect(), columns=['hash','word']).set_index('hash')

In [5]:
hashed_word

,word
hash,
605,招待客
342707,ギャンブラー
578741,フラワーカンパニーズ
196555,女子
645887,柾
478663,高位
851231,歴史の真実を求める世界連合会
987339,Finland
445743,道後温泉


In [6]:
# Tf-Idfの生成
tf = hashingTF.transform(documents)
tf.cache()

idf = IDF().fit(tf)
tf_idf_data = idf.transform(tf)

In [7]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

In [8]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors
print dt.now().strftime('%Y/%m/%d %H:%M:%S')
K = 30


# Index documents with unique IDs
corpus = tf_idf_data.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

# Cluster the documents into three topics using LDA
%time ldaModel = LDA.train(corpus, k=K)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize()) + " words):")
%time topics = ldaModel.topicsMatrix()

print dt.now().strftime('%Y/%m/%d %H:%M:%S')

2015/09/20 17:31:17
CPU times: user 6.34 ms, sys: 2.09 ms, total: 8.44 ms
Wall time: 30.8 s
Learned topics (as distributions over vocab of 1048576 words):
CPU times: user 5min 14s, sys: 6min 12s, total: 11min 26s
Wall time: 11min 53s
2015/09/20 17:43:42


In [10]:
def idx_to_word(idx):
    global temp
    res = hashed_word.ix[idx].word
    if type(res) == pd.Series:
        temp.append(res)
        return res.to_dict().values()[0]
    else:
        return res

rep_num = 20

for topic in range(K):
    print("Topic " + str(topic) + ":")
    temp_w = []
    temp_t = []
    for word in range(0, ldaModel.vocabSize()):

        top = topics[word][topic]
        if top != 0:
            #print("{}:{}".format(word, top))
            temp_w.append(word)
            temp_t.append(top)
    
    temp_w = np.array(temp_w)
    temp_t = np.array(temp_t)
    idx = np.argsort(temp_t)[::-1]
    print ','.join(map(idx_to_word, temp_w[idx[:rep_num]]))
    print temp_t[idx[:rep_num]]


Topic 0:
3D,1%,JP,Yahoo,co.jp,http://,2Z,FE,TC,WC,JavaScript,SRC,ALT,D2,分,.S,SIG,clear,Mi,GIF
[ 30498.99621439   6067.97495307   5638.31180986   4239.90976107
   3839.63866955   3620.87671019   2048.76800459   2035.55013512
   2035.55013512   2035.55013512   1903.02711354   1898.96547573
   1820.93929181   1763.1621581    1724.74815005   1688.15876657
   1613.83355369   1483.59938276   1454.82128817   1338.48860166]
Topic 1:
ディープラーニング,GPU,債務超過,原発,亜人,稼働,京セラ,演算,浜田,日本インター,推薦,村上氏,ウッドワード,ライブラリ,買い付け,ABC,DI,活用,予防医学,純資産
[ 230.26782221  222.54019498  109.0725775    86.27167445   86.10057908
   84.22603202   67.68409895   66.99081298   60.91536464   57.4006148
   57.16789412   50.24346965   50.17063652   45.16572514   43.57092785
   43.37177773   43.06492631   41.84250571   40.60449032   39.60700784]
Topic 2:
音,教授,アンプ,訴訟,スピーカー,スズキ,大学,共産党,A-10,DR,（株）,SUZUKI,さん,資本金,HONDA,板野,氏,録画,Internet,帯
[ 313.9497848   311.07373468  291.18216703  200.41036663  174.99267573
  168.83426043  162.12249119  160.4631